# Global and local scopes

In Python the **global** scope refers to the **module** scope.

The scope of a variable is normally defined by **where** it is (lexically) defined in the code.

In [1]:
a = 10
def my_func(n):
    print('global:', a)
    c = a ** n
    return c
my_func(5)

global: 10


100000

a is a global variable, and c is a local variable.

The scope of a variable is determined by where it is assigned. In particular, any variable defined (i.e. assigned a value) inside a function is local to that function, even if the variable name happens to be global too!

Remember that whenever you assign a value to a variable without having specified the variable as **global**, it is **local** in the current scope. **Moreover**, it does not matter **where** the assignment in the code takes place, the variable is considered local in the **entire** scope - Python determines the scope of objects at compile-time, not at run-time.

# Nonlocal scopes

Functions defined inside anther function can reference variables from that enclosing scope, just like functions can reference variables from the global scope.

In [2]:
def outer_func():
    x = 'hello'
    
    def inner_func():
        print(x)
    
    inner_func()
    print(x)

outer_func()

hello
hello


In [3]:
# The x variable in inner_func is a local variable which won't affect 
# the x in the outer_func scope.
def outer_func():
    x = 'hello'
    
    def inner_func():
        x = 'good'
        print(x)
    
    inner_func()
    print(x)

outer_func()

print('====nonlocal====')
# But we can change the x in the outer_func scope by adding nonlocal to the inner_func
def outer_func():
    x = 'hello'
    
    def inner_func():
        nonlocal x
        x = 'good'
        print(x)
    
    inner_func()
    print(x)

outer_func()


good
hello
====nonlocal====
good
good


# Closures

Closure: **A function** plus an **extended scope** that contains the free variables.

In [4]:
# The function outer returns a closure inner
def outer():
    x = 'python'
    print('outer x: ', hex(id(x)))
    def inner():
        print('inner x: ', hex(id(x)))
        print(x)
    return inner
fn = outer()

outer x:  0x7f0cf64aa1f0


In [5]:
fn()

inner x:  0x7f0cf64aa1f0
python


In [6]:
fn

<function __main__.outer.<locals>.inner()>

In [7]:
fn.__closure__

(<cell at 0x7f0cf146da50: str object at 0x7f0cf64aa1f0>,)

The closure is an reference pointing to a string object, which is the x, and the address of x in inner and outer are the same.

## Applications

In [8]:
# An average function

def average():
    """
    Returns the average number as you add numbers.
    """
    total = 0
    count = 0
    def add(n):
        nonlocal total, count
        total += n
        count += 1
        return total/count
    return add

In [9]:
avg = average()

In [10]:
avg(10)

10.0

In [11]:
avg(20)

15.0

In [12]:
# A timer example
from time import perf_counter

In [13]:
def itmer():
    start = perf_counter()
    def elapsed():
        return perf_counter() - start
    return elapsed

In [14]:
t = itmer()

In [15]:
t()

0.005579133998253383

In [16]:
t()

0.011497688996314537

In [17]:
# A counter function
def counter(init_value=0):
    def inc(step=1):
        nonlocal init_value
        init_value += step
        return init_value
    return inc

In [18]:
c = counter(2)

In [19]:
for i in range(10):
    print(f'Running {c(2)} times')

Running 4 times
Running 6 times
Running 8 times
Running 10 times
Running 12 times
Running 14 times
Running 16 times
Running 18 times
Running 20 times
Running 22 times


In [20]:
# Build a function to count how many times we have run some function.
def counter(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Running the {fn.__name__} {count} times')
        return fn(*args, **kwargs)
    return inner

In [21]:
def add_(a, b):
    return a + b

In [22]:
add_ = counter(add_)
add_(5, 6)

Running the add_ 1 times


11

In [23]:
add_(100, 100)

Running the add_ 2 times


200

In [24]:
# Modify the counter function to record run count for whatever function we pass in.
def counter(fn, counters):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        counters[fn.__name__] = count
        return fn(*args, **kwargs)
    return inner

In [25]:
def add(a, b):
    """
    Add two numbers or two strings.
    """
    return a + b

def mult(a, b):
    """
    Multiple two numbers or a string with a number.
    """
    return a * b

In [26]:
func_counters = {}
count_add = counter(add, func_counters)
count_mult = counter(mult, func_counters)

In [27]:
print(count_add(5,6))
print(count_add('a', 'b'))
print(count_mult(2, 3))
print(count_mult('a', 5))

11
ab
6
aaaaa


In [28]:
func_counters

{'add': 2, 'mult': 2}

# Decorators

In [35]:
def counter(fn):
    count = 0
    
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print('Function {0} was called {1} times'.format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

In [36]:
@counter
def add(a, b):
    """
    returns sum of two integers
    """
    return a + b

In [37]:
add(4, 4)

Function add was called 1 times


8

In [38]:
add(2, 3)

Function add was called 2 times


5

In [39]:
add.__name__

'inner'

The name of the function is no longer add, but instead it's the name of the inner function.

We can use wraps to make everything back to normal.

In [40]:
from functools import wraps

In [41]:
def counter(fn):
    count = 0
    
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print('Function {0} was called {1} times'.format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

@counter
def add(a, b):
    """
    returns sum of two integers
    """
    return a + b

In [42]:
add(4, 4)

Function add was called 1 times


8

In [44]:
print(add.__name__)
print(add.__doc__)

add

    returns sum of two integers
    


In [45]:
help(wraps)

Help on function wraps in module functools:

wraps(wrapped, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
    Decorator factory to apply update_wrapper() to a wrapper function
    
    Returns a decorator that invokes update_wrapper() with the decorated
    function as the wrapper argument and the arguments to wraps() as the
    remaining arguments. Default arguments are as for update_wrapper().
    This is a convenience function to simplify applying partial() to
    update_wrapper().



## Decorator application - Timer

Create a function to time how long a function takes to run a certain function.

In [67]:
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k, v) for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        print('{0}({1}) took {2:.6f}s to run.'.format(fn.__name__, args_str, elapsed))
        return result
    
    return inner

Let's write a function that calculates the n-th Fibonacci number:

`1, 1, 2, 3, 5, 8, ...`

We will implement this using two different methods:
1. recursion
2. a loop

In [68]:
# Recursion
def fib_rec(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib_rec(n-1) + fib_rec(n-2)
    
# Only time the recursion function with final result.
# It prevents from printing all recursive steps.
@timed
def fib_recursed(n):
    return fib_rec(n)

In [82]:
fib_recursed(35)

fib_recursed(35) took 2.139661s to run.


9227465

In [77]:
# Loop

def fib_loop(n):
    f0 = 0
    f1 = 1
    for i in range(2, n+1):
        f0, f1 = f1, f1 + f0
    return f1

In [79]:
fib_loop(10)

55

In [80]:
@timed
def fib_looped(n):
    return fib_loop(n)

In [83]:
fib_looped(35)

fib_looped(35) took 0.000014s to run.


9227465

The loop method is more faster. That's because the recursive one will re-calculate intermediate variables every time.